# Tagging and Extraction Using OpenAI functions

# Tagging

- We have seen the LLM, given a function description, select aguments from the input text generate a structured output forming a function call

- More generally, the LLM can evaluate the input text and generate structured output

texts --> LLM (Structure description) --> {sentiment: positive, language: Spanish}

texts --> LLM (Structure description) --> {first ane: lang, last anme: chain, language: Python}

## in short, instead of asking LLM to reason through the texts, we are asking LLM to reformat the structure of the texts.

### Course overview

### This notebook only introducing tagging and extractions, but they have many usage beyond, such as:

1) Tracking and organizing Data
- particularly useful when dealing with large datasets or document stores. Good for info retreival and search.

2) Context management
- In chatbot applications, tagging can be used to track the context of user interactions.
- for example, if a user asks multiple questions, we can tag responses with topics or categories, like "product information" or "pricing", which helps the system maintain context and follow-up appropriately.


### next notebook targets these questions:
#### routing and task Management
#### better search and filtering
#### Customization and personalization

### the chain's results eventually would depend on how many times it has been practiced. Sometimes, the first few times you throw a chain into the model, the results are not that good. For example, if you ask for both Chinese and English version of some summary, it would only show English at first. Or if you ask for a price, it would give you a wrong number. However, after you execute the lines more often, the "more correct and comprehensive answer" would eventually show up. 

In [1]:
!pip install langchain openai
!pip install python-dotenv
!pip install pydantic
!pip install tiktoken
!pip install langchain_community

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
OPENAI_API_KEY = openai.api_key

In [3]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [4]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [5]:
convert_pydantic_to_openai_function(Tagging)

<ipython-input-5-ae58ff6334b6>:1: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  convert_pydantic_to_openai_function(Tagging)


{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'properties': {'sentiment': {'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language'],
  'type': 'object'}}

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [7]:
model = ChatOpenAI(api_key= OPENAI_API_KEY)

<ipython-input-7-215443698e81>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(api_key= OPENAI_API_KEY)


In [8]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [9]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [10]:
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [11]:
tagging_chain = prompt | model_with_functions

In [12]:

tagging_chain.invoke({"input": "I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"en"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 108, 'total_tokens': 118, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9eed23ec-2214-4a4c-ae17-6dd11a84d88d-0')

In [13]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"neg","language":"it"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2f5cf01f-ce13-4b81-9102-5297bd969da9-0')

In [14]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# jsonoutput helps with the format

In [15]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

# add as one of the paramters

In [16]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}

In [17]:
tagging_chain.invoke({"input": "I love langchain"})

{'sentiment': 'pos', 'language': 'en'}

In [18]:
tagging_chain.invoke({"input": "Langchain was created by Harrison Chase. \
It is an open-source developer framework that helps users build applications powered by LLMs."})

{'sentiment': 'pos', 'language': 'en'}

# Put tagging together

### In summary, the essential parts are:

- Tagging class definition;
- convert_pydantic_to_openai_function(Tagging_Function)
- tagging_functions = [convert_pydantic_to_openai_function(Tagging)]
- prompt = ChatPromptTemplate.from_messages([
    ("system", "XXXXXXXXXX"),
    ("user", "{input}")
])
- model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

- define the tagging Chaing using prompt, model_with_functions, json pretifier....
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

##### Essential imports, besides the usual ones:

- from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
- from langchain.prompts import ChatPromptTemplate
- from langchain.chat_models import ChatOpenAI
- from langchain.prompts import ChatPromptTemplate
- from langchain.chat_models import ChatOpenAI
- from typing import List
- from pydantic import BaseModel, Field
- from langchain.utils.openai_functions import convert_pydantic_to_openai_function

# add as one of the paramters

In [19]:
# now, modify the tagging definition slightly.
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    objects: str = Field(description="capture the objects")
    summary: str = Field(description="summarize the texts in 10 words or less.")
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    explainatory: str = Field(description="explain why the text is the given sentiment")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [20]:
convert_pydantic_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'properties': {'objects': {'description': 'capture the objects',
    'type': 'string'},
   'summary': {'description': 'summarize the texts in 10 words or less.',
    'type': 'string'},
   'sentiment': {'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'explainatory': {'description': 'explain why the text is the given sentiment',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['objects', 'summary', 'sentiment', 'explainatory', 'language'],
  'type': 'object'}}

In [21]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

# add as one of the paramters

In [22]:
model = ChatOpenAI(api_key= OPENAI_API_KEY)

In [23]:
tagging_chain.invoke({"input": "The movie Substance stands out as an original film, \
a refreshing departure from the usual Hollywood fare. It embodies the kind of innovative \
storytelling that the industry should embrace more often. It shows that unique ideas still have a place on the big screen. \
Importantly, it doesn't need a sequel or prequel—its story is complete as is, and adding more would not be super interesting.\
It’s remarkable that Substance was made with a modest budget of just $17.5 million, yet it delivers a visually and narratively \
compelling experience. Surprisingly, its box office earnings of only $2 million fall short of expectations, \
considering the buzz around it. Despite this, the movie has garnered higher ratings on critical review platforms than on Google Reviews, \
indicating stronger support from critics than the general audience.\
Viewers, at least, I, would much rather Hollywood take note of films like Substance—bold, original projects that push creative \
boundaries—and continue to invest in similarly daring ventures."})

{'objects': 'movie Substance',
 'summary': 'Praise for originality, innovation, and storytelling; modest budget and earnings',
 'sentiment': 'pos',
 'explainatory': 'The text praises the movie for its originality, innovation, and storytelling, as well as its modest budget and critical acclaim.',
 'language': 'en'}

## Extraction

Extraction is similar to tagging, but used for extracting **multiple** pieces of information.

#### and you can define your own function to teach the chain how to summarize and tag the correct information retrived from the texts.

In [24]:
from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [25]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")
    # what we areally are about is to identify this "Person" field so that it can be reconstructured in real examples.

In [26]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'properties': {'people': {'description': 'List of info about people',
    'items': {'description': 'Information about a person.',
     'properties': {'name': {'description': "person's name", 'type': 'string'},
      'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
       'description': "person's age"}},
     'required': ['name', 'age'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['people'],
  'type': 'object'}}

This is converting your Information model (Pydantic) into a function definition that openai's model can work with. Essentially, it translates the Python data structures into something the model can understand to extract information from input text.

The convert_pydantic_to_openai_function(Information) translates the schema of the Information model so that it can be used to extract structured data in the form of Information when invoked in the chain.

In [27]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

model.bind: This binds the open model (GPT-4 or something else) to the specific functions defined using convert_pydantic_to_openai_function. So it is specifying that the model should call the "Information" function when extracting data.




invoke: The invoke method calls the model with an input (in this case, "Joe is 30, his mom is Martha"). The model is instructed to extract information about people using the function you just defined (Information).

In [28]:
extraction_model.invoke("Joe is 30, his mom is Martha")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 95, 'total_tokens': 116, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-43a79bba-15e0-4323-b532-2c1919e215d1-0')

In [29]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [30]:
extraction_chain = prompt | extraction_model

In [31]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 112, 'total_tokens': 133, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-68c23fd8-b4d0-43d2-98f4-6e0680d05918-0')

In [32]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()
# could just use json directly.

In [33]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]}

In [34]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [35]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [36]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha, and her age is 65"})

# better format

## Put things together Extractions


In [37]:
from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")


In [38]:
from pydantic import BaseModel, Field, EmailStr, HttpUrl, conint
from typing import List, Optional
from enum import Enum
from datetime import date, datetime

In [39]:
!pip install pydantic[email]
# interesting.....

In [40]:
# code source: https://python.langchain.com/v0.1/docs/use_cases/extraction/how_to/examples/

class Gender(str, Enum):
    MALE = "male"
    FEMALE = "female"
    OTHER = "other"

class Address(BaseModel):
    street: str = Field(description="street address")
    city: str = Field(description="city name")
    zipcode: str = Field(description="zip code")
    country: str = Field(description="country name")

class Job(BaseModel):
    company_name: str = Field(description="Company where the person works")
    job_title: str = Field(description="lob title or position")

class Person(BaseModel):
    name: str = Field(description="Person's full name", min_length=2)
    age: Optional[conint(ge=0, le=150)] = Field(description="Person's age")
    email: Optional[EmailStr] = Field(description="Person's email address")
    gender: Optional[Gender] = Field(description="Person's gender")
    address: Optional[Address] = Field(description="Person's address")
    jobs: Optional[List[Job]] = Field(description="List of jobs held by the person")
    birth_date: Optional[date] = Field(description="Person's birth date")
    friends: Optional[List[str]] = Field(description="List of friends' names")
    relatives: Optional[List[str]] = Field(description="List of relatives' names")
    is_employed: bool = Field(description="Employment status of the person", default=False)
    created_at: datetime = Field(default_factory=datetime.utcnow)

class Information(BaseModel):
    people: List[Person] = Field(description="List of people information")


In [41]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': '',
 'parameters': {'properties': {'people': {'description': 'List of people information',
    'items': {'properties': {'name': {'description': "Person's full name",
       'minLength': 2,
       'type': 'string'},
      'age': {'anyOf': [{'maximum': 150, 'minimum': 0, 'type': 'integer'},
        {'type': 'null'}],
       'description': "Person's age"},
      'email': {'anyOf': [{'format': 'email', 'type': 'string'},
        {'type': 'null'}],
       'description': "Person's email address"},
      'gender': {'anyOf': [{'enum': ['male', 'female', 'other'],
         'title': 'Gender',
         'type': 'string'},
        {'type': 'null'}],
       'description': "Person's gender"},
      'address': {'anyOf': [{'properties': {'street': {'description': 'street address',
           'title': 'Street',
           'type': 'string'},
          'city': {'description': 'city name',
           'title': 'City',
           'type': 'string'},
          'zipcode':

In [42]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]

extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [43]:
extraction_model.invoke("Joe is 30, his mom is Martha. He ")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30,"relatives":["Martha"]}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 355, 'total_tokens': 373, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-47f4bb4e-9860-47e5-93e5-6308331326c4-0')

In [44]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])


In [45]:
extraction_chain = prompt | extraction_model

In [46]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha. He works at Google as \
a data scientist. He has 3 friends, and they are Tom, Jack, and Tim. \
He was born in 1987."})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30,"jobs":[{"company_name":"Google","job_title":"Data Scientist"}],"birth_date":"1987","friends":["Tom","Jack","Tim"],"relatives":["Martha"]}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 403, 'total_tokens': 449, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bb677c70-a121-4961-a6cd-c06f33268e37-0')

In [47]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})


In [48]:
json_parser = JsonKeyOutputFunctionsParser(key_name="people")


In [49]:
extraction_chain = prompt | extraction_model | json_parser


In [50]:
result = extraction_chain.invoke({"input": "Joe is 30, his mom is Martha. He works at Google as \
a data scientist. He has 3 friends, and they are Tom, Jack, and Tim. \
He was born in 1987."})



In [51]:
result

[{'name': 'Joe',
  'age': 30,
  'jobs': [{'company_name': 'Google', 'job_title': 'Data Scientist'}],
  'birth_date': '1987',
  'friends': ['Tom', 'Jack', 'Tim'],
  'relatives': ['Martha']}]

In [52]:
# from typing import Optional
# class Person(BaseModel):
#     """Information about a person."""
#     name: str = Field(description="person's name")
#     age: Optional[int] = Field(description="person's age")

# class Information(BaseModel):
#     """Information to extract."""
#     people: List[Person] = Field(description="List of info about people")


convert_pydantic_to_openai_function(Information)

extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

extraction_model.invoke("Joe is 30, his mom is Martha")

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

extraction_chain = prompt | extraction_model

extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

# extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]

## Doing it for real

We can apply tagging to a larger body of text.

For example, let's load this blog post and extract tag information from a sub-set of the text.

In [53]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [54]:
doc = documents[0]

In [55]:
page_content = doc.page_content[:10000]

In [56]:
print(page_content[:1000])

# since this is a web, the contents can be very scarce







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general

In [57]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [58]:
overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [59]:
tagging_chain.invoke({"input": page_content})

{'summary': 'Overview of LLM powered autonomous agents, including components like Planning, Memory, and Tool Use. Discusses techniques like Task Decomposition and Self-Reflection in agent systems.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection'}

In [60]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

In [61]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=paper_extraction_function,
    function_call={"name":"Info"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [62]:
extraction_chain.invoke({"input": page_content})

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

In [63]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article.

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [64]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [65]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)',
  'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct (Yao et al. 2023)', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)',
  'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)',
  'author': 'Laskin et al. 2023'}]

In [66]:
extraction_chain.invoke({"input": "hi"})

[{'title': 'Paper 1', 'author': 'Author 1'},
 {'title': 'Paper 2', 'author': 'Author 2'}]

In [67]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [68]:
splits = text_splitter.split_text(doc.page_content)

In [69]:
len(splits)

15

In [70]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [71]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [72]:
print(splits[0])

LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general probl

In [73]:
from langchain.schema.runnable import RunnableLambda

In [74]:
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [75]:
prep.invoke("hi")

[{'input': 'hi'}]

In [76]:
chain = prep | extraction_chain.map() | flatten

In [79]:
# chain.invoke(doc.page_content)
# exceeds rate limit, but the idea is it would extract all the papers that were mentioned in the webpage.

## Put these all together.

#### Try to extract some information from a PDF uplaoded here.
 - The PDF would be about the cafe Chain Pret. The intersting thing about Pret is that it does not produce that much good coffee compared to starbucks but they have comparable prices. Most likely due to its membership subscriptions and healthy sandwitch options, it survives. this PDF also has Chinese in them. English and Chinese have the same words.


In [84]:
!pip install pypdf

In [85]:
from langchain.document_loaders import PyPDFLoader

pdf_path = "Pret.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load() # it's actully a list

In [107]:

all_contents = "\n".join([doc.page_content for doc in documents])

len(all_contents)


9589

In [110]:
class Overview(BaseModel):
    """Tag the piece of text with particular info."""
    objects: str = Field(description="capture the objects")
    summary: str = Field(description="summarize the texts in 10 words or less.")
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    explainatory: str = Field(description="explain why the text is the given sentiment")
    question: str = Field(description="Answer the question, what is Pret's competing advantage?")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [111]:
overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [112]:
tagging_chain.invoke({"input": all_contents})

{'objects': '1983 when two college friends, Sinclair Beecham and Julian Metcalfe',
 'summary': 'The story of Pret A Manger begins with Sinclair Beecham and Julian Metcalfe in 1983.',
 'sentiment': 'neutral',
 'explainatory': 'The text provides historical information without expressing any positive or negative sentiment.',
 'question': "What is Pret's competing advantage?",
 'language': 'en'}

In [114]:
tagging_chain.invoke({"input": all_contents})

{'objects': "1983, Sinclair Beecham, Julian Metcalfe, Hampstead, London, McDonald's, New York City, 2000, Bridgepoint",
 'summary': 'Extracted papers mentioned in the article about the history of Pret A Manger',
 'sentiment': 'neutral',
 'explainatory': 'The text provides a brief history of Pret A Manger, mentioning key events, locations, and partnerships.',
 'question': "What is Pret's competing advantage?",
 'language': 'en'}

In [190]:
class Food_item(BaseModel):
    """Information about food mentioned in the menu."""
    title: str
    price: float
    description: Optional[str]

class Menu(BaseModel):
    """Information to extract"""
    menus: List[Food_item]

class Milestones(BaseModel):
    """Information to extract"""
    event: str
    date: str
    description: str

class History(BaseModel):
    """Information to extract"""
    histories: List[Milestones]

In [199]:
pdf_extraction_function = [
    convert_pydantic_to_openai_function(Menu)
]
extraction_model = model.bind(
    functions= pdf_extraction_function,
    function_call={"name":"Menu"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="menus")

In [174]:
# prompt = PromptTemplate(template="Extract the menus from the following text:\n{input}")


In [200]:
extraction_chain.invoke({"input": all_contents})

[{'title': 'Classic Tuna & Cucumber', 'price': 5.5, 'description': None},
 {'title': 'Chicken Caesar & Bacon Baguette',
  'price': 7.0,
  'description': None},
 {'title': 'Ham & Cheese Baguette', 'price': 6.0, 'description': None},
 {'title': 'Egg Salad Sandwich', 'price': 5.0, 'description': None},
 {'title': 'Chicken & Avocado Salad', 'price': 8.0, 'description': None},
 {'title': 'Chef’s Italian Salad', 'price': 7.5, 'description': None},
 {'title': 'Tuna Nicoise Salad', 'price': 8.5, 'description': None},
 {'title': 'Super Greens & Grains Salad', 'price': 8.5, 'description': None},
 {'title': 'Falafel & Hummus Wrap', 'price': 6.0, 'description': None},
 {'title': 'Chicken & Bacon Caesar Wrap', 'price': 7.0, 'description': None},
 {'title': 'Mexican Chicken Avocado Wrap', 'price': 7.5, 'description': None},
 {'title': 'Macaroni Cheese Prosciutto', 'price': 7.0, 'description': None},
 {'title': 'Chicken & Mozzarella Toastie', 'price': 6.5, 'description': None},
 {'title': 'Spinach & 

In [201]:
template = """A article will be passed to you. Extract the menus from the PDF,

I want you to do this to both the English texts and the Chinese texts. please make sure the
price is correct.

Format them nicely, preferably each menu item is on one single line.

一篇文章将会传递给你。请从PDF中提取菜单信息。

要求你对英文和中文文本中的菜单都进行提取，并确保价格的正确性。

请将提取的内容进行整齐的格式化，最好每个菜单项占一行。. """

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [202]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="menus")

In [182]:
# template = """A article will be passed to you. Extract the menus from the PDF,

# I want you to do this to both the English texts and the Chinese texts. please make sure the
# price is correct.

# Format them nicely, preferably each menu item is on one single line.

# Please also give the Chinese version of the menus too.

# """

# prompt = ChatPromptTemplate.from_messages([
#     ("system", template),
#     ("human", "{input}")
# ])
# extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="menus")
# extraction_chain.invoke({"input": all_contents})

In [203]:
extraction_chain.invoke({"input": all_contents})
# and Chinese menus are not showing

[{'title': 'Classic Tuna & Cucumber', 'price': 5.5, 'description': None},
 {'title': 'Chicken Caesar & Bacon Baguette',
  'price': 7.0,
  'description': None},
 {'title': 'Ham & Cheese Baguette', 'price': 6.0, 'description': None},
 {'title': 'Egg Salad Sandwich', 'price': 5.0, 'description': None},
 {'title': 'Chicken & Avocado Salad', 'price': 8.0, 'description': None},
 {'title': 'Chef’s Italian Salad', 'price': 7.5, 'description': None},
 {'title': 'Tuna Nicoise Salad', 'price': 8.5, 'description': None},
 {'title': 'Super Greens & Grains Salad', 'price': 8.5, 'description': None},
 {'title': 'Falafel & Hummus Wrap', 'price': 6.0, 'description': None},
 {'title': 'Chicken & Bacon Caesar Wrap', 'price': 7.0, 'description': None},
 {'title': 'Mexican Chicken Avocado Wrap', 'price': 7.5, 'description': None},
 {'title': 'Macaroni Cheese Prosciutto', 'price': 7.0, 'description': None},
 {'title': 'Chicken & Mozzarella Toastie', 'price': 6.5, 'description': None},
 {'title': 'Spinach & 

In [204]:
extraction_chain.invoke({"input": "hello"})


# it still generates some stuff,
# why?
# The reason you're getting that output (a list of menu items with titles like "Appetizers,"
# "Main Course," and "Desserts") from the input "hi" is because the model, when bound to
# functions and asked to generate specific structured data like Menu, will try to fabricate
# or guess an answer based on the structure it knows, even if the input doesn't provide relevant information.

[{'title': 'Grilled Salmon',
  'price': 15.99,
  'description': 'Freshly grilled salmon fillet with a side of steamed vegetables'},
 {'title': 'Chicken Alfredo',
  'price': 12.99,
  'description': 'Creamy fettuccine pasta with grilled chicken strips'},
 {'title': 'Cheeseburger',
  'price': 9.99,
  'description': 'Juicy beef patty topped with melted cheese, lettuce, and tomato'}]

### History:

In [215]:
pdf_extraction_function = [
    convert_pydantic_to_openai_function(History)
]
extraction_model = model.bind(
    functions= pdf_extraction_function,
    function_call={"name":"History"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="histories")

In [216]:
extraction_chain.invoke({"input": all_contents})

[{'event': 'Pret A Manger founded by Sinclair Beecham and Julian Metcalfe',
  'date': '1983',
  'description': 'First shop opened in Hampstead, London in 1984, offering fresh sandwiches and salads made with high-quality ingredients'},
 {'event': "McDonald's acquires 33% stake in Pret A Manger",
  'date': '1996',
  'description': 'Provided capital for further expansion, leading to the first international store opening in New York City in 2000'},
 {'event': 'Bridgepoint acquires majority stake in Pret A Manger',
  'date': '2008',
  'description': 'Allowed for more growth and innovation, expanding the menu to include vegetarian and vegan options'}]

In [236]:
template = """A article will be passed to you. Extract the menus from the PDF,

I want you to summarize the milestones in Pret's history. Each milestone should
include the event, date, and a short description.

Format them nicely, preferably each milestone is on one single line.

一篇文章将会传递给你。请把里面的中文从PDF中提取菜单信息。

我希望你总结Pret历史中的重要里程碑。每个里程碑应包括事件、日期和简短描述。

请将提取的内容进行整齐的格式化，最好每个里程碑占一行。

请把中文版本的histories也做出来 """

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="histories")

In [237]:
extraction_chain.invoke({"input": all_contents})
# and Chinese contents are not showing

[{'event': 'Pret A Manger founded by Sinclair Beecham and Julian Metcalfe',
  'date': '1983',
  'description': 'First shop opened in Hampstead, London in 1984 offering fresh, natural food'},
 {'event': "McDonald's acquires 33% stake in Pret A Manger",
  'date': '1996',
  'description': 'Provided capital for further expansion, leading to the first international store in New York City in 2000'},
 {'event': 'Bridgepoint acquires majority stake in Pret A Manger',
  'date': '2008',
  'description': 'Enabled further growth and innovation, including menu expansion with more vegetarian and vegan options'}]

# Milestones (using runnable)

## using runnable usually has better results.


In [220]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [222]:
splits = text_splitter.split_text(all_contents)
len(splits)

3

In [223]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [226]:
# flatten([[1, 2], [3, 4]])

In [227]:
print(splits[0][:20])

The
History
of
Pret



In [229]:
from langchain.schema.runnable import RunnableLambda

In [238]:
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [239]:
prep.invoke("hi")

[{'input': 'hi'}]

In [240]:
chain = prep | extraction_chain.map() | flatten

In [241]:
chain.invoke(all_contents)
# no Chinese either, but much better English results.

[{'event': 'Founding of Pret A Manger by Sinclair Beecham and Julian Metcalfe',
  'date': '1983',
  'description': 'Gap in the market for fresh, natural food in London, first shop opened in 1984 with a focus on fresh sandwiches and salads'},
 {'event': "McDonald's acquires 33% stake in Pret A Manger",
  'date': '1996',
  'description': 'Provided capital for further expansion, partnership allowed international reach with first store in New York City in 2000'},
 {'event': 'Bridgepoint acquires majority stake in Pret A Manger',
  'date': '2008',
  'description': 'Private equity firm acquisition enabled growth and innovation, expanded menu to include vegetarian and vegan options'},
 {'event': 'Founding of Pret A Manger by Sinclair Beecham and Julian Metcalfe',
  'date': '1983',
  'description': 'Two university friends noticed the lack of fresh and natural food options in the market, leading to the idea of creating a place offering high-quality sandwiches, salads, and coffee. The first Pret

In [242]:
# chain.invoke(all_contents)
extraction_chain.invoke({"input": all_contents})
# slightly worse than "chain.invoke(all_contents)"

[{'event': 'Founding of Pret A Manger by Sinclair Beecham and Julian Metcalfe',
  'date': '1983',
  'description': 'Two college friends noticed a gap in the market for fresh, natural food and opened the first shop in Hampstead, London in 1984.'},
 {'event': "McDonald's acquires a 33% stake in Pret A Manger",
  'date': '1996',
  'description': "McDonald's investment provided capital for further expansion, leading to the opening of the first international store in New York City in 2000."},
 {'event': 'Bridgepoint acquires a majority stake in Pret A Manger',
  'date': '2008',
  'description': 'Private equity firm acquisition supports growth and innovation, including menu expansion to include more vegetarian and vegan options.'}]

In [243]:
chain = prep | extraction_chain.map() | flatten
chain.invoke(all_contents)
# getting better while you keep sending requests....
# interesting observation: as the chain request increases, the results get better.

[{'event': 'Pret A Manger founded by Sinclair Beecham and Julian Metcalfe',
  'date': '1983',
  'description': 'Gap in the market for fresh, natural food in London, first shop opened in 1984 with focus on high-quality ingredients and quick service'},
 {'event': "McDonald's acquires 33% stake in Pret A Manger",
  'date': '1996',
  'description': 'Provided capital for further expansion, leading to international presence with the first store in New York City in 2000'},
 {'event': 'Bridgepoint acquires majority stake in Pret A Manger',
  'date': '2008',
  'description': 'Enabled further growth, expansion of menu to include vegetarian and vegan options to cater to changing consumer demands'},
 {'event': 'Founding of Pret A Manger by Sinclair Beecham and Julian Metcalfe',
  'date': '1983',
  'description': 'Two university friends noticed the lack of fresh and natural food options in London, leading to the creation of a place offering high-quality sandwiches, salads, and coffee. The first Pre

# Menu (using runnable)

## to get the menu results, you have to rerun the menu part above and then get the defualt results.

In [161]:
from langchain.schema.runnable import RunnableLambda

# results are much better!!!

In [162]:
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [163]:
prep.invoke("hi")

[{'input': 'hi'}]

In [165]:
chain = prep | extraction_chain.map() | flatten

In [166]:
chain.invoke(all_contents)
# extraction_chain.invoke({"input": all_contents})


[{'title': 'Pret A Manger Sandwiches',
  'price': 8.99,
  'description': 'Freshly made sandwiches with high-quality ingredients'},
 {'title': 'Pret A Manger Salads',
  'price': 7.99,
  'description': 'Healthy and delicious salads made daily'},
 {'title': 'Pret A Manger Coffee',
  'price': 2.99,
  'description': 'High-quality coffee made to order'},
 {'title': 'Classic Tuna & Cucumber', 'price': 5.5, 'description': None},
 {'title': 'Chicken Caesar & Bacon Baguette',
  'price': 7.0,
  'description': None},
 {'title': 'Ham & Cheese Baguette', 'price': 6.0, 'description': None},
 {'title': 'Egg Salad Sandwich', 'price': 5.0, 'description': None},
 {'title': 'Chicken & Avocado Salad', 'price': 8.0, 'description': None},
 {'title': 'Chef’s Italian Salad', 'price': 7.5, 'description': None},
 {'title': 'Tuna Nicoise Salad', 'price': 8.5, 'description': None},
 {'title': 'Super Greens & Grains Salad', 'price': 8.5, 'description': None},
 {'title': 'Falafel & Hummus Wrap', 'price': 6.0, 'descr

In [169]:
extraction_chain.invoke({"input": all_contents})


[{'title': 'Sandwiches and Baguettes',
  'price': 5.5,
  'description': 'Classic Tuna & Cucumber'},
 {'title': 'Sandwiches and Baguettes',
  'price': 6.5,
  'description': 'Classic Tuna & Cucumber'},
 {'title': 'Sandwiches and Baguettes',
  'price': 7,
  'description': 'Chicken Caesar & Bacon Baguette'},
 {'title': 'Sandwiches and Baguettes',
  'price': 8.5,
  'description': 'Chicken Caesar & Bacon Baguette'},
 {'title': 'Sandwiches and Baguettes',
  'price': 6,
  'description': 'Ham & Cheese Baguette'},
 {'title': 'Sandwiches and Baguettes',
  'price': 7,
  'description': 'Ham & Cheese Baguette'},
 {'title': 'Sandwiches and Baguettes',
  'price': 5,
  'description': 'Egg Salad Sandwich'},
 {'title': 'Sandwiches and Baguettes',
  'price': 6,
  'description': 'Egg Salad Sandwich'},
 {'title': 'Salads', 'price': 8, 'description': 'Chicken & Avocado Salad'},
 {'title': 'Salads', 'price': 9.5, 'description': 'Chicken & Avocado Salad'},
 {'title': 'Salads', 'price': 7.5, 'description': 'Che

In [170]:
chain = prep | extraction_chain.map() | flatten

In [171]:
# chain = prep | extraction_chain.map() | flatten
chain.invoke(all_contents)

[{'title': 'Pret A Manger Coffee',
  'price': 3.5,
  'description': 'Freshly brewed coffee made from high-quality beans'},
 {'title': 'Pret A Manger Sandwich',
  'price': 5.99,
  'description': 'Freshly prepared sandwich with a variety of fillings'},
 {'title': 'Pret A Manger Salad',
  'price': 7.99,
  'description': 'Healthy salad made with fresh ingredients'},
 {'title': 'Classic Tuna & Cucumber', 'price': 5.5, 'description': None},
 {'title': 'Chicken Caesar & Bacon Baguette', 'price': 7, 'description': None},
 {'title': 'Ham & Cheese Baguette', 'price': 6, 'description': None},
 {'title': 'Egg Salad Sandwich', 'price': 5, 'description': None},
 {'title': 'Chicken & Avocado Salad', 'price': 8, 'description': None},
 {'title': 'Chef’s Italian Salad', 'price': 7.5, 'description': None},
 {'title': 'Tuna Nicoise Salad', 'price': 8.5, 'description': None},
 {'title': 'Super Greens & Grains Salad', 'price': 8.5, 'description': None},
 {'title': 'Falafel & Hummus Wrap', 'price': 6, 'descr

In [ ]:

# class Overview(BaseModel):
#     """Overview of a section of text."""
#     summary: str = Field(description="Provide a concise summary of the content.")
#     language: str = Field(description="Provide the language that the content is written in.")
#     keywords: str = Field(description="Provide keywords related to the content.")
# class Paper(BaseModel):
#     """Information about papers mentioned."""
#     title: str
#     author: Optional[str]
# class Info(BaseModel):
#     """Information to extract"""
#     papers: List[Paper]